# Applegate Mechanism Calculator
### A Hands-on Notebook Based on Völschow et al. (2016)

In this notebook I implement and explore the energy requirements of the Applegate mechanism for explaining eclipse timing variations (ETV) in close binary systems.

I follow the formalism presented in [Völschow et al. (2016)](https://ui.adsabs.harvard.edu/abs/2016A%26A...587A..34V/abstract).

I will:

1. Define the relevant stellar and orbital parameters.
2. Implement three different models for the Applegate mechanism:
   - Thin-shell approximation (Tian et al. 2009, as quoted by Völschow et al. 2016)
   - Constant-density model
   - Two-zone model
3. Compare the required energy to the energy budget of the secondary star.

## 1. Physical Constants and Units

All equations in Völschow et al. (2016) are expressed in cgs units. I will use [Astropy-Constants](https://docs.astropy.org/en/stable/constants/index.html) to obtain physical constants and convert them to cgs.

I also adopt the IAU 2015 nominal solar effective temperature:
$T_\odot = 5772 ~\mathrm{K}.$

In [3]:
from astropy.constants import G, M_sun, R_sun, L_sun
from astropy import units as u
import numpy as np
import math

G_cgs = G.cgs.value
M_sun_cgs = M_sun.cgs.value
R_sun_cgs = R_sun.cgs.value
L_sun_cgs = L_sun.cgs.value

day_to_sec = u.day.to(u.s)
year_to_sec = u.year.to(u.s)

T_sun = 5772  # Kelvin

## 2. Input Parameters for the Secondary Star and the Binary

I consider a close post-common-envelope binary (PCEB) consisting of:

- A white dwarf primary, and
- A low-mass main-sequence secondary star with:
  - mass $M_{\mathrm{sec}}$ (in units of $M_\odot$),
  - radius $R_{\mathrm{sec}}$ (in units of $R_\odot$),
  - effective temperature $T_{\mathrm{sec}}$ (in Kelvin),
  - luminosity $L_{\mathrm{sec}}$ (in units of $L_\odot$).

The binary is characterized by:

- binary separation $a_{\mathrm{bin}}$ (in units of $R_\odot$),
- orbital period $P_{\mathrm{bin}}$ (in days),
- modulation period $P_{\mathrm{mod}}$ of the ETV signal (in days or years),
- observed relative period change $\Delta P / P_{\mathrm{bin}}$ (dimensionless),
  or equivalently a timing semi-amplitude $A$ in seconds.

## 3. Thin-Shell Approximation (Tian et al. 2009)

In the thin-shell approximation, Tian et al. (2009) (quoted as Eq. (7) in Völschow et al. 2016)
derived an approximate relation between the required energy and the observed timing variations:

$$
\frac{\Delta E}{E_{\mathrm{sec}}}
= 0.233
\left( \frac{M_{\mathrm{sec}}}{M_{\odot}} \right)^{3}
\left( \frac{R_{\mathrm{sec}}}{R_{\odot}} \right)^{-10}
\left( \frac{T_{\mathrm{sec}}}{6000~\mathrm{K}} \right)^{-4}
\left( \frac{a_{\mathrm{bin}}}{R_{\odot}} \right)^{4}
\left( \frac{\Delta P}{\mathrm{s}} \right)^{2}
\left( \frac{P_{\mathrm{mod}}}{\mathrm{yr}} \right)^{-1}
$$

Here $\Delta P$ is the absolute period change in seconds, and $P_{\mathrm{mod}}$ is the modulation period in years. $E_{\mathrm{sec}}$ is the energy produced by the secondary over one modulation period.

In [6]:
def thin_shell_energy_ratio(M_sec, R_sec, T_sec, a_bin, delta_P_sec, P_mod_yr):
    ratio = 0.233 * (M_sec**3) * (R_sec**-10) * (T_sec / 6000)**(-4) * (a_bin**4) * (delta_P_sec**2) * (P_mod_yr**-1)
    return ratio

## 4. Constant-Density Model (Völschow et al. 2016, Eq. 22)

Assuming a constant density profile for the secondary star, Völschow et al. (2016) derive for the required energy (their Eq. (22)):

$$
\frac{\Delta E}{E_{\mathrm{sec}}}
\simeq 1.1 \times 10^{7}
\left( \frac{\Delta P}{P_{\mathrm{bin}}} \right)
\left( \frac{a_{\mathrm{bin}}}{R_{\odot}} \right)^{2}
\left( \frac{M_{\mathrm{sec}}}{M_{\odot}} \right)^{2}
\left( \frac{R_{\mathrm{sec}}}{R_{\odot}} \right)^{-3}
\left( \frac{P_{\mathrm{mod}}}{\mathrm{yr}} \right)^{-1}
\left( \frac{L_{\mathrm{sec}}}{L_{\odot}} \right)^{-1}
$$

This expression already includes the division by the energy available over one modulation period, so it directly yields $\Delta E / E_{\mathrm{sec}}$.

In [8]:
def constant_density_energy_ratio(deltaP_over_Pbin, a_bin, M_sec, R_sec, P_mod_yr, L_sec):
    ratio = 1.1e7 * deltaP_over_Pbin * (a_bin**2) * (M_sec**2) * (R_sec**-3) * (P_mod_yr**-1) * (L_sec**-1)
    return ratio

## 5. Two-Zone Model (Völschow et al. 2016, Eq. 35–37)

The two-zone model represents the secondary star as an inner core and an outer shell with different
densities. The exchange of angular momentum between these two regions changes the stellar quadrupole
moment and therefore the orbital period. The required energy is given by:

$$
\frac{\Delta E}{E_{\mathrm{sec}}}
= k_{1}\,
\frac{M_{\mathrm{sec}} R_{\mathrm{sec}}^{2}}
     {P_{\mathrm{bin}}^{2}\, P_{\mathrm{mod}}\, L_{\mathrm{sec}}}
\left(
    1 \pm
    \sqrt{
        1 -
        k_{2}\, G\,
        \frac{
            a_{\mathrm{bin}}^{2}\,
            M_{\mathrm{sec}}\,
            P_{\mathrm{bin}}^{2}
        }{
            R_{\mathrm{sec}}^{5}
        }
        \frac{\Delta P}{P_{\mathrm{bin}}}
    }
\right)^{2}
$$


The constants $k_1$ and $k_2$ are structural coefficients of the two-zone approximation.

- $k_1$ is a structural coefficient that measures how efficiently a given exchange of angular momentum between core and shell can change the stellar quadrupole moment. It depends on the relative moments of inertia of the core and the shell.

- $k_2$ determines how strongly a change in the quadrupole moment couples back into the orbital period. It effectively controls the strength of the interaction between the stellar deformation and the binary orbit.

For low-mass main-sequence secondaries typical of post-common-envelope binaries, Völschow et al. (2016) obtain:

$$
k_1 = 0.133, \qquad k_2 = 3.42
$$

The term under the square root is what Völschow et al. (2016) refer to as the Applegate parameter:

$$
A = k_{2}\, G\,
\frac{
    a_{\mathrm{bin}}^{2}\,
    M_{\mathrm{sec}}\,
    P_{\mathrm{bin}}^{2}
}{
    R_{\mathrm{sec}}^{5}
}
\left(
    \frac{\Delta P}{P_{\mathrm{bin}}}
\right)
$$

A physical solution exists only if $(A \le 1)$. If $(A > 1)$, the Applegate mechanism cannot produce the observed period variations.

In [10]:
k1 = 0.133
k2 = 3.42

def two_zone_energy_ratio(M_sec, R_sec, L_sec, a_bin, P_bin_days, deltaP_over_Pbin, P_mod_days, G_value=G_cgs):
    
    a_cgs = a_bin * R_sun_cgs
    M_cgs = M_sec * M_sun_cgs
    R_cgs = R_sec * R_sun_cgs
    L_cgs = L_sec * L_sun_cgs
    
    P_bin_sec = P_bin_days * day_to_sec
    P_mod_sec = P_mod_days * day_to_sec
    
    # Applegate parameter A (their Eq. 37)
    A = k2 * G_value * (a_cgs**2 * P_bin_sec**2 * M_cgs / R_cgs**5) * deltaP_over_Pbin

    if A > 1.0:
        return None, A

    prefactor = k1 * (M_cgs * R_cgs**2) / (P_bin_sec**2 * P_mod_sec * L_cgs)
    ratio = prefactor * (1.0 - math.sqrt(1.0 - A))**2

    return ratio, A

## 6. Application to Example Systems from Völschow et al. (2016)

We now apply the three analytical models

1. Thin-shell approximation (Tian et al. 2009, Eq. 7),
2. Constant-density model (Völschow et al. 2016, Eq. 22),
3. Two-zone model (Völschow et al. 2016, Eq. 35–37),

to three well-studied systems from Völschow et al. (2016): **NN Ser**, **HW Vir**, and **QS Vir**.

For each system we adopt the stellar and orbital parameters listed in their Tables 1 and 2, and the
period modulation parameters from Table 3 (i.e. modulation period and $\Delta P / P_{\mathrm{bin}}$). We then compute $\Delta E / E_{\mathrm{sec}}$ for all three models and compare them to the values reported in Table 4 of the paper.

In [12]:
# Parameters for three systems from Völschow et al. (2016)
# NN Ser, HW Vir, QS Vir
systems = {
    "NN Ser": {"M_sec": 0.111,
               "R_sec": 0.149,
               "T_sec": 2920,
               "L_sec": 0.00147,
               "a_bin": 0.934,
               "P_bin_days": 0.130,     # days
               "P_mod_yr": 15.482,      # years
               "deltaP_over_Pbin": 7.1e-7},
    
    "HW Vir": {"M_sec": 0.142,
               "R_sec": 0.175,
               "T_sec": 3084,
               "L_sec": 0.003,
               "a_bin": 0.860,
               "P_bin_days": 0.117,     # days
               "P_mod_yr": 55,          # years
               "deltaP_over_Pbin": 4.1e-6},
    
    "QS Vir": {"M_sec": 0.43,
               "R_sec": 0.42,
               "T_sec": 3100.0,
               "L_sec": 0.0146,
               "a_bin": 1.27,
               "P_bin_days": 0.151,     # days
               "P_mod_yr": 16.99,       # years
               "deltaP_over_Pbin": 1.0e-6}}

In [13]:
for name, p in systems.items():
    M = p["M_sec"]
    R = p["R_sec"]
    T = p["T_sec"]
    L = p["L_sec"]
    a = p["a_bin"]
    
    P_bin_days = p["P_bin_days"]
    P_bin_sec = P_bin_days * day_to_sec
    
    P_mod_yr = p["P_mod_yr"]
    P_mod_days = P_mod_yr * 365.25
    
    deltaP_over_Pbin = p["deltaP_over_Pbin"]
    delta_P_sec = deltaP_over_Pbin * P_bin_sec

    ratio_thin = thin_shell_energy_ratio(M_sec=M, 
                                         R_sec=R, 
                                         T_sec=T, 
                                         a_bin=a, 
                                         delta_P_sec=delta_P_sec, 
                                         P_mod_yr=P_mod_yr)

    ratio_const = constant_density_energy_ratio(deltaP_over_Pbin=deltaP_over_Pbin, 
                                                a_bin=a, 
                                                M_sec=M, 
                                                R_sec=R, 
                                                P_mod_yr=P_mod_yr, 
                                                L_sec=L)

    ratio_two, A = two_zone_energy_ratio(M_sec=M, 
                                         R_sec=R, 
                                         L_sec=L, 
                                         a_bin=a, 
                                         P_bin_days=P_bin_days, 
                                         deltaP_over_Pbin=deltaP_over_Pbin, 
                                         P_mod_days=P_mod_days)

    print(f"\u2022 {name}")
    print(f"  {'A parameter':20s}: {A:.3f}")
    print(f"  {'Thin-shell':20s}: dE/E_sec ~= {ratio_thin:.2f}")
    print(f"  {'Constant-density':20s}: dE/E_sec ~= {ratio_const:.2f}")
    print(f"  {'Two-zone':20s}: dE/E_sec ~= {ratio_two:.2f}")

• NN Ser
  A parameter         : 0.159
  Thin-shell          : dE/E_sec ~= 3.29
  Constant-density    : dE/E_sec ~= 1115.03
  Two-zone            : dE/E_sec ~= 62.72
• HW Vir
  A parameter         : 0.361
  Thin-shell          : dE/E_sec ~= 6.06
  Constant-density    : dE/E_sec ~= 760.59
  Two-zone            : dE/E_sec ~= 110.25
• QS Vir
  A parameter         : 0.012
  Thin-shell          : dE/E_sec ~= 0.04
  Constant-density    : dE/E_sec ~= 178.50
  Two-zone            : dE/E_sec ~= 0.71


## 7. Interpretation of the Results

Using the analytical expressions reproduced in this notebook, we compute $\Delta E / E_{\mathrm{sec}}$ for the systems NN Ser, HW Vir, and QS Vir. Our results reproduce the values reported in Table 4 of Völschow et al. (2016) to within a few percent, confirming that our implementation is correct.

### Consistency with Völschow et al. (2016)

- **NN Ser**  
  - Thin-shell: 3.29  
  - Constant-density: 1115.03  
  - Two-zone: 62.72  
  These values are in excellent agreement with the published values  
  (3.3, 1100, 64).  

- **HW Vir**  
  - Thin-shell: 6.06 
  - Constant-density: 760.59  
  - Two-zone: 110.25
  These values are again consistent with the reported values  
  (6.0, 720, 108).  

- **QS Vir**  
  - Thin-shell: 0.04  
  - Constant-density: 178.50  
  - Two-zone: 0.71  
  Matching the values  
  (0.040, 170, 0.71).

These comparisons verify that our thin-shell, constant-density, and two-zone implementations reproduce the analytical models described by Völschow et al. (2016)

---

## 8. Astrophysical Interpretation

$\Delta E / E_{\mathrm{sec}}$ measures the energy required to produce the observed period modulation relative to the total energy generated by the secondary during one modulation cycle.

- **ΔE/E_sec ≪ 1** → energetically very easy; Applegate mechanism is viable  
- **ΔE/E_sec ∼ 1** → borderline case; energetically possible  
- **ΔE/E_sec ≫ 1** → energetically impossible; would require more energy than the star can supply  

### NN Ser  
Even in the most realistic approximation (two-zone), $\Delta E / E_{\mathrm{sec}} \approx 60$ meaning the required energy exceeds the available energy by almost two orders of magnitude. The Applegate mechanism is energetically unfeasible for NN Ser, strongly supporting a third-body interpretation for its timing variations.

### HW Vir  
Similarly, HW Vir yields $\Delta E/E_{\mathrm{sec}} \approx 110$ again far above unity. The Applegate mechanism is rejected on energetic grounds for HW Vir as well.

### QS Vir  
QS Vir is fundamentally different: its two-zone value $\Delta E/E_{\mathrm{sec}} \approx 0.7$ is of order unity. This means the magnetic quadrupole mechanism is energetically viable for QS Vir and cannot be ruled out. This system remains a borderline case in which both Applegate-type magnetic cycles and third-body scenarios should be considered.

---

In summary, our results confirm the findings of Völschow et al. (2016): most systems require too much energy for an Applegate-type mechanism, except for a few borderline cases such as QS Vir.

## Application to NY Vir using Esmer et al. (2023)

As a final step, I applied the same implementation to the **NY Vir** system using the parameters reported by [Esmer et al. (2023)](https://ui.adsabs.harvard.edu/abs/2023MNRAS.525.6050E/abstract). Their study also evaluates the required energy for the **Thin–shell**, **Constant–density**, and **Two–zone** models.  

Here, I repeat the calculation with my own implementation and compare the results.

In [16]:
# Parameters for NY Vir from Esmer et al. (2023)

M2   = 0.13       # M_sun
R2   = 0.155      # R_sun
T2   = 2048       # K

a_bin = 0.77           # R_sun
P_bin = 0.1010159690   # days

P_mod_days = 8127.15    # days
A_OC_sec   = 35.1       # s

In [17]:
L2 = (R2**2) * (T2 / T_sun)**4

P_mod_sec = P_mod_days * day_to_sec
P_mod_yr  = P_mod_days / 365.25

deltaP_over_P = 2.0 * math.pi * A_OC_sec / P_mod_sec
deltaP_sec    = deltaP_over_P * P_bin * day_to_sec


ratio_thin = thin_shell_energy_ratio(M2, R2, T2, a_bin, deltaP_sec, P_mod_yr)

ratio_const = constant_density_energy_ratio(deltaP_over_P, a_bin, M2, R2, P_mod_yr, L2)

ratio_two, A = two_zone_energy_ratio(M2, R2, L2, a_bin, P_bin, deltaP_over_P, P_mod_days)

print(f"\u2022 NY Vir (Esmer et al. 2023) ")
print(f"  {'A parameter':20s}: {A:.3f}")
print(f"  {'Thin-shell':20s}: dE/E_sec ~= {ratio_thin:.3f}")
print(f"  {'Constant-density':20s}: dE/E_sec ~= {ratio_const:.3f}")
print(f"  {'Two-zone':20s}: dE/E_sec ~= {ratio_two:.3f}")

• NY Vir (Esmer et al. 2023) 
  A parameter         : 0.028
  Thin-shell          : dE/E_sec ~= 0.559
  Constant-density    : dE/E_sec ~= 1097.172
  Two-zone            : dE/E_sec ~= 10.035


Using the parameters reported by Esmer et al. (2023) for NY Vir, our implementation reproduces the published values with good agreement:

- Thin–shell model: $\Delta E / E_{\mathrm{sec}} = 0.559$  
- Constant–density model: $\Delta E / E_{\mathrm{sec}} = 1097.17$  
- Two–zone model: $\Delta E / E_{\mathrm{sec}} = 10.035$

These values match the results presented in Esmer et al. (2023):  
$\Delta E / E_{\mathrm{sec}} = 0.559$, $1101.75$, and $10.07$ for the thin–shell, constant–density, and two–zone models, respectively. The small numerical differences between our results and those reported by Esmer et al. (2023) are expected and most likely arise from the specific choices of physical constants used in the calculations, in particular the adopted solar temperature and related luminosity scaling.

From a physical standpoint, the interpretation follows directly:

- The constant–density model predicts extremely large energy requirements ($\sim 10^3$), well beyond the secondary star's capability.  
- The two–zone model also yields a value $(\sim 10$), meaning the star would need to devote more than ten times its available energy to drive the modulation - not physically plausible.  
- Only the thin–shell model gives a value below unity, but still large ($\sim 0.56$), implying that more than half of the secondary’s luminosity would need to be channelled into quadrupole moment oscillations.

Therefore, even in the most optimistic scenario (thin–shell approximation), the energy requirement remains uncomfortably high.  
This supports the conclusion of Esmer et al. (2023) that the Applegate mechanism is unlikely to be the primary cause of the observed ETV modulation in NY Vir.

## Conclusion

In this notebook, I implemented three formulations of the Applegate mechanism—the thin–shell, constant–density, and two–zone models—following the formalisms presented by Völschow et al. (2016) and Esmer et al. (2023). By applying these models to the post–common-envelope binaries NN Ser, HW Vir, QS Vir, and NY Vir, I successfully reproduced the energy budget values reported in the literature. This agreement validates the accuracy of the implementation and confirms the reliability of the code for investigating the Applegate mechanism across different systems.

From an astrophysical standpoint, for NN Ser and HW Vir all models require $\Delta E/E_{\mathrm{sec}} \gg 1$, confirming that the mechanism is energetically impossible. QS Vir remains the only borderline case: its two–zone value of $\sim 0.7$ indicates that magnetic quadrupole variations
cannot be ruled out.

Applying the same analysis to NY Vir using the parameters from Esmer et al. (2023), the results again show that the required energy—especially $\Delta E/E_{\mathrm{sec}} \approx 10$ from the two–zone model—is far above what the secondary can supply.

In summary, while QS Vir may marginally allow Applegate-type activity, the mechanism is not energetically viable for NN Ser, HW Vir, or NY Vir, reinforcing the conclusion that magnetic cycles cannot explain their long-term eclipse timing variations.